In [1]:
# OPTIONAL: Load the "autoreload" eX_orig[alias]tension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

%matplotlib inline

In [2]:
# Import Biopython modules to interact with KEGG
from Bio import SeqIO
from Bio.KEGG import REST
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas

import pandas as pd
import numpy as np
from functools import reduce

from src.visualization.plots import HR_GENE_SET, NHEJ_GENE_SET, FANCONI_ANEMIA_GENE_SET
from src.config import get_common_barcodes, HOME_PATH, get_hussmann_supplementary_xlsx


TOP_GENES_LIMIT = 400

In [3]:
final_genes = pd.read_csv("../outlier_detection/artifacts/final_gene_list.tsv", sep="\t", usecols=[0, 6], skiprows=1)[1:].rename(columns={"Measure": "Gene"})
final_genes.head()

,Gene,Pseudo-control
1,Atp6v1g1,False
2,H2ac18,False
3,Metap2,False
4,Xrcc5,False
5,H2ax,False


In [4]:
hussmann_repair_genes = pd.read_excel(get_hussmann_supplementary_xlsx(4), sheet_name="Table S4", skiprows=1)["Gene"].str.lower().str.capitalize()

In [5]:
top_genes = final_genes["Gene"].iloc[:TOP_GENES_LIMIT]
top_genes

1      Atp6v1g1
2        H2ac18
3        Metap2
4         Xrcc5
5          H2ax
         ...   
396        Smc3
397       Brca1
398       Sec13
399        Bop1
400     Gm10024
Name: Gene, Length: 400, dtype: object

In [6]:
# top_hits = reduce(np.union1d, (top_genes, HR_GENE_SET, NHEJ_GENE_SET, FANCONI_ANEMIA_GENE_SET, POLQ))
# len(top_hits)

In [7]:
filtered_barcodes = pd.read_csv(get_common_barcodes(), sep="\t")
filtered_barcodes.head()

,Target,Alias,Gene,Barcode,Filtered,Counts,Corr_Within,Corr_Between
0,T1,MB01,0610009B22Rik,0610009B22Rik-1,0,2247.0,0.802124,0.719819
1,T1,MB01,0610009B22Rik,0610009B22Rik-2,0,2792.0,0.861144,0.841499
2,T1,MB01,0610009B22Rik,0610009B22Rik-3,0,3907.0,0.894923,0.900597
3,T1,MB01,0610009B22Rik,0610009B22Rik-4,0,3757.0,0.886405,0.779313
4,T1,MB01,0610009B22Rik,0610009B22Rik-5,0,2549.0,0.920184,0.844981


In [8]:
good_barcodes = filtered_barcodes[filtered_barcodes["Filtered"].isin([0, 5])]
good_barcodes_any_T = good_barcodes[good_barcodes["Target"].isin(["T1", "T2", "T3"])]
good_barcodes_any_T

,Target,Alias,Gene,Barcode,Filtered,Counts,Corr_Within,Corr_Between
0,T1,MB01,0610009B22Rik,0610009B22Rik-1,0,2247.0,0.802124,0.719819
1,T1,MB01,0610009B22Rik,0610009B22Rik-2,0,2792.0,0.861144,0.841499
2,T1,MB01,0610009B22Rik,0610009B22Rik-3,0,3907.0,0.894923,0.900597
3,T1,MB01,0610009B22Rik,0610009B22Rik-4,0,3757.0,0.886405,0.779313
4,T1,MB01,0610009B22Rik,0610009B22Rik-5,0,2549.0,0.920184,0.844981
...,...,...,...,...,...,...,...,...
536759,T3,MB06,ccdc198,ccdc198-5,0,2585.0,0.957342,0.959532
536760,T3,MB06,mei-04,Mei4-1,0,3218.0,0.872856,0.888647
536761,T3,MB06,mei-04,Mei4-2,0,2200.0,0.834222,0.763714
536762,T3,MB06,mei-04,Mei4-3,0,4083.0,0.821211,0.926477


In [17]:
# https://www.genome.jp/brite/mmu03400

brite_file = REST.kegg_get("br:mmu03400").read()

repair_genes = []
current_section = None
current_subsection = None

subsections_of_interest = [
    "B  DSBR (double strand breaks repair)",
    "B  Other factors with a suspected DNA repair function",
    "B  Check point factors",
    "B  TLS (translesion DNA synthesis) factors",
    # "B  SSBR (single strand breaks repair)"
]

for line in brite_file.rstrip().split("\n"):
    section = line[:1]
    if section == "A":
        current_section = line[1:]
        print(current_section)

    if section == "B":
        current_subsection = line
        print(current_subsection)
    
    if current_section ==  "Eukaryotic type":
        if current_subsection in subsections_of_interest:
            if ";" in line:
                repair_genes.append(line.split()[2][:-1])
print("There are {} repair pathway repair_genes in KEGG".format(len(repair_genes)))

print("There is an overlap of {} Hussmann genes with the KEGG repair genes".format(len(np.intersect1d(repair_genes, hussmann_repair_genes))))

Eukaryotic type
B  SSBR (single strand breaks repair)
B  DSBR (double strand breaks repair)
B  TLS (translesion DNA synthesis) factors
B  Check point factors
B  Other factors with a suspected DNA repair function
Prokaryotic type
B  SSBR (single strand breaks repair)
B  DSBR (double strand breaks repair)
B  TLS (translesion DNA synthesis) factors
B  Other factors with a suspected DNA repair function
There are 278 repair pathway repair_genes in KEGG
There is an overlap of 121 Hussmann genes with the KEGG repair genes


In [18]:
top_hits = np.unique(reduce(np.union1d, (top_genes, repair_genes)))
len(top_hits)

616

In [11]:
top_hit_barcodes = good_barcodes_any_T[good_barcodes_any_T["Gene"].isin(top_hits)]
top_hit_barcode_counts = top_hit_barcodes.groupby(["Gene", "Barcode"]).size().groupby("Gene").size() 
top_hit_barcode_counts = top_hit_barcode_counts[top_hit_barcode_counts>=1]
top_hit_barcode_counts.shape

(591,)

In [12]:
"There is an overlap of {} Hussmann genes with our top hits".format(len(np.intersect1d(top_hits, hussmann_repair_genes)))

'There is an overlap of 128 Hussmann genes with our top hits'

In [13]:
top_hit_barcode_counts.value_counts().sort_index()

1    118
2    103
3    133
4    103
5    134
dtype: int64

Redo sgRNA filter analysis without read count filter and with read count filter in parallel (can think about this)

In [14]:
NUM_GOOD_BARCODES = 3

# barcodes that are in T2 and T3 for top hits
top_hit_barcodes = good_barcodes_any_T[good_barcodes_any_T["Gene"].isin(top_hits)]

# barcodes that are good in any (1) or both (2) replicates
top_hit_barcode_counts_per_target = top_hit_barcodes.groupby(["Gene", "Target", "Barcode"]).size()
barcodes_good_in_both_replicates = top_hit_barcode_counts_per_target[top_hit_barcode_counts_per_target >= 1]

# barcodes that are good in both replicates in X target sites, where X in [1, 2, 3]
barcodes_counts_across_both_replicates = barcodes_good_in_both_replicates.groupby(["Gene", "Barcode"]).size()
barcodes_counts_across_both_replicates = barcodes_counts_across_both_replicates[barcodes_counts_across_both_replicates >= 1]
barcodes_counts_across_both_replicates

# genes that have at least 3 good barcodes
(barcodes_counts_across_both_replicates.groupby("Gene").size() >= NUM_GOOD_BARCODES).sum()

370

In [15]:
genes_with_X_barcodes = barcodes_counts_across_both_replicates.groupby("Gene").size() >= NUM_GOOD_BARCODES
genes_with_X_barcodes = genes_with_X_barcodes[genes_with_X_barcodes].index.to_list()
print("There are {} genes with at least {} good barcodes".format(len(genes_with_X_barcodes), NUM_GOOD_BARCODES))

repair_genes_with_X_barcodes = np.intersect1d(genes_with_X_barcodes, repair_genes)
print("Of that, {} are known repair genes".format(len(repair_genes_with_X_barcodes)))

print("And {} are not".format(len(genes_with_X_barcodes) - len(repair_genes_with_X_barcodes)))

There are 370 genes with at least 3 good barcodes
Of that, 212 are known repair genes
And 158 are not
